# Analyze results of Timeloop/Accelergy on NeRF

In [72]:
import json
import os
import pandas as pd
import regex as re

In [36]:
!ls profile_results

eyeriss_like_onchip_compression_results.json
eyeriss_like_results.json
results.json
simba_like_results.json
simple_weight_stationary_results.json


## Load Results from the Profile notebok

In [69]:
def load_results(results_path: str = "profile_results/results.json") -> dict:
    """ Load results from Profile NeRF.ipynb notebook back into original shape. """
    with open(results_path, "r") as f:
        results = json.load(f)
    print(f"Loaded results from {results_path}")

    for arch, arch_result_dict in results.items():
        arch_results_path = arch_result_dict["results"]
        assert os.path.exists(arch_results_path), f"Could not find {arch_results_path} for {arch}"

        with open(arch_results_path, "r") as f:
            arch_result_dict["results"] = json.load(f)    
        print(f"Populated {arch} results from {arch_results_path}")
        
    return results

profile_results = load_results()

Loaded results from profile_results/results.json
Populated eyeriss_like results from /home/workspace/notebooks/profile_results/eyeriss_like_results.json
Populated eyeriss_like_onchip_compression results from /home/workspace/notebooks/profile_results/eyeriss_like_onchip_compression_results.json


## Analyze

In [82]:
# Compute the actual area as the summary got it wrong
area_regex = r"Area.*:\s(\d+.\d+)"

for arch, arch_results in profile_results.items():
    layer_summary = arch_results["layer_summary"]
    for layer_id in layer_summary:
        # Read all areas from the timeloop stats file
        stats_fname = layer_summary[layer_id]["stats_fname"]
        with open(stats_fname, "r") as f:
            stats = [l.strip() for l in f.readlines()]

        area_lines = [line for line in stats[:-50] if re.match(area_regex, line)]
        areas = [re.search(area_regex, line).group(1) for line in area_lines]
        areas = [float(area) for area in areas]
        total_area = sum(areas)

        layer_summary[layer_id]["actual_areas"] = areas
        layer_summary[layer_id]["actual_total_area"] = total_area

In [85]:
# Massage results into dataframes
layer_dfs = {}
all_summary = {}
drop_cols = ["name", "stats_fname", "actual_areas"]

for arch, arch_results in profile_results.items():
    all_summary[arch] = arch_results["summary"]

    # Load layer results into dataframe
    df_layer = pd.DataFrame.from_dict(arch_results["layer_summary"], orient="index")
    df_layer = df_layer.drop(columns=drop_cols)
    df_layer.index.name = "layer_id"
    layer_dfs[arch] = df_layer
    
df_summary = pd.DataFrame.from_dict(all_summary, orient="index")

In [86]:
layer_dfs["eyeriss_like"]

,total_area,total_energy,total_cycle,num,energy,area,cycle,gflops,utilization,edp,shape,actual_total_area
layer_id,,,,,,,,,,,,
1,0.0,11.42,10752,1,11.42,0.0,10752,190.48,0.57,0.12300,"{'C': 63, 'M': 256, 'N': 64}",1375449.72
2,0.0,260.40,458752,7,37.20,0.0,65536,127.75,0.38,2.44000,"{'C': 256, 'M': 256, 'N': 64}",1375449.72
6,0.0,59.17,59392,1,59.17,0.0,59392,175.72,0.52,3.51000,"{'C': 319, 'M': 256, 'N': 64}",1375449.72
9,0.0,2.84,2048,1,2.84,0.0,2048,15.97,0.05,0.00581,"{'C': 256, 'M': 1, 'N': 64}",1375449.72
11,0.0,315.54,36224,1,315.54,0.0,36224,127.77,0.38,11.40000,"{'C': 283, 'M': 128, 'N': 64}",1375449.72
12,0.0,1.56,1024,1,1.56,0.0,1024,47.81,0.14,0.00160,"{'C': 128, 'M': 3, 'N': 64}",1375449.72


In [87]:
layer_dfs["eyeriss_like_onchip_compression"]

,total_area,total_energy,total_cycle,num,energy,area,cycle,gflops,utilization,edp,shape,actual_total_area
layer_id,,,,,,,,,,,,
1,0.0,12.91,10752,1,12.91,0.0,10752,190.48,0.57,0.13900,"{'C': 63, 'M': 256, 'N': 64}",907149.12
2,0.0,41.10,65536,1,41.10,0.0,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12
3,0.0,41.10,65536,1,41.10,0.0,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12
4,0.0,41.10,65536,1,41.10,0.0,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12
5,0.0,41.10,65536,1,41.10,0.0,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12
6,0.0,58.37,59392,1,58.37,0.0,59392,175.72,0.52,3.47000,"{'C': 319, 'M': 256, 'N': 64}",907149.12
7,0.0,41.10,65536,1,41.10,0.0,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12
8,0.0,41.10,65536,1,41.10,0.0,65536,127.75,0.38,2.69000,"{'C': 256, 'M': 256, 'N': 64}",907149.12
9,0.0,2.58,2048,1,2.58,0.0,2048,15.97,0.05,0.00529,"{'C': 256, 'M': 1, 'N': 64}",907149.12
